In [6]:
#!pip install transformers

In [13]:
!pip install transformers==4.3.0

     |████████████████████████████████| 1.8 MB 5.8 MB/s            
     |████████████████████████████████| 2.2 MB 26.0 MB/s            
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [6]:
!pip install --upgrade pip

  Using cached pip-21.3.1-py3-none-any.whl (1.7 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.2.2
    Uninstalling pip-21.2.2:
      Successfully uninstalled pip-21.2.2


In [3]:
!pip install sentencepiece

In [4]:
!pip install accelerate

     |████████████████████████████████| 106 kB 6.9 MB/s eta 0:00:01


In [5]:
!pip install  xlrd==1.1.0

     |████████████████████████████████| 108 kB 6.5 MB/s            


In [7]:
!pip install --upgrade ipywidgets

  Using cached ipywidgets-7.8.1-py2.py3-none-any.whl (124 kB)
  Using cached comm-0.1.4-py3-none-any.whl (6.6 kB)
  Using cached widgetsnbextension-3.6.6-py2.py3-none-any.whl (1.6 MB)
  Using cached jupyterlab_widgets-1.1.7-py3-none-any.whl (295 kB)
  Using cached parso-0.7.1-py2.py3-none-any.whl (109 kB)
  Attempting uninstall: parso
    Found existing installation: parso 0.8.3
    Uninstalling parso-0.8.3:
      Successfully uninstalled parso-0.8.3


In [1]:
import pandas as pd
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
import torch

# Load data into pandas DataFrame
data_df = pd.read_excel("StudentEssays.xlsx")

# Initialize T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# Move the model to the CUDA device if available
if torch.cuda.is_available():
    model.to("cuda")

# Define a list of concepts to predict
concepts_to_predict = ["potential energy", "kinetic energy", "Law of Conservation of Energy"]

# Define possible outcome labels
outcome_labels = ["Acceptable", "Unacceptable", "Insufficient", "Not Found"]

# Create a list to store predictions as dictionaries
predictions_list = []

# Iterate through each row of text data
for index, row in data_df.iterrows():
    text = row['Essay']  # Assuming the text content is in column 'Essay'

    # Initialize predictions dictionary for this row
    predictions = {}

    # Iterate through each concept to predict
    for concept in concepts_to_predict:
        # Define a template for classification
        template = f"According to the following essay, is the student's definition of {concept} Acceptable, Unacceptable, Insufficient, or Not Found? Only use one of these labels for outputs\n{text}"
        # Prepare the input by replacing placeholders
        formatted_input = template
        # Tokenize and classify the text
        input_ids = tokenizer(formatted_input, return_tensors="pt", padding=True, truncation=True).input_ids.to("cuda" if torch.cuda.is_available() else "cpu")
        outputs = model.generate(input_ids, max_length=128)
        decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)  # Remove special tokens

        # Store the prediction in the dictionary
        predictions[concept] = next((label for label in outcome_labels if label.lower() in decoded_output.lower()), "Unknown")

        if predictions[concept] == "Unknown":
          print(len(decoded_output))
          with open('output.txt', 'w') as f:
            f.write(decoded_output)

    # Append the predictions to the list
    predictions_list.append(predictions)

# Convert the list of dictionaries to a DataFrame
predictions_df = pd.DataFrame(predictions_list)

# # Print the predictions
# print(predictions_df)
# Set options to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Print the predictions
print(predictions_df)


   Law of Conservation of Energy kinetic energy potential energy
0                     Acceptable      Not Found        Not Found
1                      Not Found      Not Found        Not Found
2                     Acceptable   Insufficient     Insufficient
3                   Insufficient   Insufficient     Insufficient
4                     Acceptable   Insufficient     Insufficient
5                      Not Found   Insufficient     Insufficient
6                     Acceptable      Not Found        Not Found
7                      Not Found   Insufficient     Insufficient
8                      Not Found      Not Found     Insufficient
9                     Acceptable      Not Found     Insufficient
10                     Not Found   Insufficient     Insufficient
11                     Not Found      Not Found        Not Found
12                     Not Found   Insufficient     Insufficient
13                    Acceptable   Insufficient     Insufficient
14                    Ac

In [9]:
pip show transformers

Name: transformers
Version: 3.0.2
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /Users/gyuhoshim/opt/anaconda3/envs/conda_env/lib/python3.6/site-packages
Requires: dataclasses, filelock, numpy, packaging, regex, requests, sacremoses, sentencepiece, tokenizers, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [10]:
ls /Users/gyuhoshim/opt/anaconda3/envs/conda_env/lib/python3.6/site-packages/transformers

__init__.py
__main__.py
__pycache__/
activations.py
another_try.py
benchmark/
benchmark_utils.py
commands/
configuration_albert.py
configuration_auto.py
configuration_bart.py
configuration_bert.py
configuration_camembert.py
configuration_ctrl.py
configuration_distilbert.py
configuration_electra.py
configuration_encoder_decoder.py
configuration_flaubert.py
configuration_gpt2.py
configuration_longformer.py
configuration_marian.py
configuration_mmbt.py
configuration_mobilebert.py
configuration_openai.py
configuration_reformer.py
configuration_retribert.py
configuration_roberta.py
configuration_t5.py
configuration_transfo_xl.py
configuration_utils.py
configuration_xlm.py
configuration_xlm_roberta.py
configuration_xlnet.py
convert_albert_original_tf_checkpoint_to_pytorch.py
convert_bart_original_pytorch_checkpoint_to_pytorch.py
convert_bert_original_tf_checkpoint_to_pytorch.py
convert_bert_pytorch_checkpoint_to_original_tf.py
convert_dialogpt_original_pytorch_checkpoint_to_pytorch.py
conver

In [5]:
# Update the original DataFrame with the predictions
data_df["PE"] = predictions_df["potential energy"]
data_df["KE"] = predictions_df["kinetic energy"]
data_df["LCE"] = predictions_df["Law of Conservation of Energy"]

# Save the modified DataFrame to the same Excel file, overwriting the original file
data_df.to_excel("Result.xlsx", index=False)